https://sessions.thinkful.com/Jason-Tavonia

https://colab.research.google.com/drive/1kqkgyq_VLeuDNAasojnYHdUrC6TBoaiz

## Day 29 Lecture 1 Assignment

In this assignment, we will learn about decision trees. We will use the Chicago salary dataset loaded below.

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from math import log2

In [0]:
chicago = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Current_Employee_Names__Salaries__and_Position_Titles.csv')

In [0]:
chicago.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"ALLISON, PAUL W",LIEUTENANT,FIRE,F,Salary,NaN,$107790.00,NaN
1,"BRUNO, KEVIN D",SERGEANT,POLICE,F,Salary,NaN,$104628.00,NaN
2,"COOPER, JOHN E",LIEUTENANT-EMT,FIRE,F,Salary,NaN,$114324.00,NaN
3,"CRESPO, VILMA I",STAFF ASST,LAW,F,Salary,NaN,$76932.00,NaN
4,"DOLAN, ROBERT J",SERGEANT,POLICE,F,Salary,NaN,$111474.00,NaN


To simplify this problem, we will limit our model to only salary employees. Create a new dataset that does not contain any hourly employees.

In [0]:
chicago.drop(chicago.loc[chicago['Salary or Hourly'] == 'Hourly'].index, axis=0, inplace=True)
chicago

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"ALLISON, PAUL W",LIEUTENANT,FIRE,F,Salary,NaN,$107790.00,NaN
1,"BRUNO, KEVIN D",SERGEANT,POLICE,F,Salary,NaN,$104628.00,NaN
2,"COOPER, JOHN E",LIEUTENANT-EMT,FIRE,F,Salary,NaN,$114324.00,NaN
3,"CRESPO, VILMA I",STAFF ASST,LAW,F,Salary,NaN,$76932.00,NaN
4,"DOLAN, ROBERT J",SERGEANT,POLICE,F,Salary,NaN,$111474.00,NaN
...,...,...,...,...,...,...,...,...
32653,"ZYGOWICZ, PETER J",POLICE OFFICER,POLICE,F,Salary,NaN,$90024.00,NaN
32654,"ZYLINSKA, KATARZYNA",POLICE OFFICER,POLICE,F,Salary,NaN,$48078.00,NaN
32655,"ZYMANTAS, MARK E",POLICE OFFICER,POLICE,F,Salary,NaN,$87006.00,NaN
32656,"ZYRKOWSKI, CARLO E",POLICE OFFICER,POLICE,F,Salary,NaN,$93354.00,NaN


Next, we will look at the count of all values for both job titles and department to ensure that we don't use features that are too sparse in our model.

In [0]:
# answer below:
print('Unique Job Title values total:', chicago['Job Titles'].nunique())

print(chicago['Job Titles'].value_counts())

print('Unique Department values total:', chicago['Department'].nunique())

print(chicago['Department'].value_counts())

Unique Job Title values total: 954
POLICE OFFICER                            9393
FIREFIGHTER-EMT                           1424
SERGEANT                                  1118
POLICE OFFICER (ASSIGNED AS DETECTIVE)     845
FIREFIGHTER                                564
                                          ... 
WATER QUALITY MANAGER                        1
GIS MANAGER                                  1
MANAGER OF CERTIFICATION / COMPLIANCE        1
DIR OF NUTRITION                             1
HATE CRIME VICTIM ADVOCATE                   1
Name: Job Titles, Length: 954, dtype: int64
Unique Department values total: 36
POLICE                   12941
FIRE                      4798
OEMC                       832
PUBLIC LIBRARY             708
AVIATION                   556
FINANCE                    532
HEALTH                     512
TRANSPORTN                 396
WATER MGMNT                366
LAW                        361
CITY COUNCIL               348
FAMILY & SUPPORT           3

Choose the between department and job title and use the variable with the smallest number of values to for one hot encoding. Additionally, create dummy variables for full ot part-time.

In [0]:
# answer below:
dummy_cols = ['Full or Part-Time', 'Department']
dummies = pd.get_dummies(chicago[dummy_cols], drop_first=True)

Remove all irrelevant columns (Name, Job Titles, Salary or Hourly, Typical Hours, Hourly Rate)

In [0]:
# answer below:
irrelevant_columns = ['Name', 'Job Titles', 'Salary or Hourly', 'Typical Hours', 'Hourly Rate', 'Department', 'Full or Part-Time']

chicago.drop(irrelevant_columns, axis=1, inplace=True)

Check that none of the remaining columns are of object type and convert them to numeric if they are of object type.

In [0]:
# answer below:
chicago.select_dtypes('object')

chicago['Annual Salary'] = pd.to_numeric(chicago['Annual Salary'].str.replace('$', ''))
chicago.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24775 entries, 0 to 32657
Data columns (total 1 columns):
Annual Salary    24775 non-null float64
dtypes: float64(1)
memory usage: 387.1 KB


Split the data into a test and train sample. Use annual salary as the dependent variable. 20% of the data should be assigned to the test sample.

In [0]:
# answer below:
X_train, X_test, y_train, y_test = train_test_split(dummies, chicago, test_size=.2)

Generate a regession decision tree using `DecisionTreeRegressor` in sklearn. Fit the model on the training set and calculate the score for both train and test.

In [0]:
# answer below:
d_tree = DecisionTreeRegressor()
d_tree.fit(X_train, y_train)

print('Train score:', d_tree.score(X_train, y_train))
print('Test score:', d_tree.score(X_test, y_test))


Train score: 0.14201045117176603
Test score: 0.16327469758818136


# Day 29 Lecture 2 Assignment
# In this assignment, we will learn about entropy and information gain in the ID3 algorithm.

In [0]:
tennis = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/tennis_decision.csv')

In [0]:
tennis

,Day,Outlook,Temp.,Humidity,Wind,Decision
0,1,Sunny,Hot,High,Weak,No
1,2,Sunny,Hot,High,Strong,No
2,3,Overcast,Hot,High,Weak,Yes
3,4,Rain,Mild,High,Weak,Yes
4,5,Rain,Cool,Normal,Weak,Yes
5,6,Rain,Cool,Normal,Strong,No
6,7,Overcast,Cool,Normal,Strong,Yes
7,8,Sunny,Mild,High,Weak,No
8,9,Sunny,Cool,Normal,Weak,Yes
9,10,Rain,Mild,Normal,Weak,Yes


# Write a function to compute entropy given an input of a sequence of probabilities.

In [0]:
import scipy.stats

In [0]:
def ent(data):
    """Calculates entropy of the passed `pd.Series`
    """
    data = data.value_counts()           # counts occurrence of each value
    entropy = scipy.stats.entropy(data)  # get entropy from counts
    return entropy


In [0]:
ent(tennis)

0.0

#Aggregate the tennis decision table for each value of each column. Start with Outlook below. Compute the weighted mean of the entropy for outlook (the weighted mean of the yes decision and the no decision).

# Compute the weighted mean of the entropy for temperature, humidity and wind as well and decide based on these values which should be the first variable chosen for a split.

Participants (2)

https://sessions.thinkful.com/Jason-Tavonia
10:05 AM
Jason Randall Smith joined 
Jason Randall Smith joined the stage 
Tavonia Evans joined 
Tavonia Evans joined the stage 
Jason Randall SmithJason Randall Smith10:12 AM
https://colab.research.google.com/drive/1kqkgyq_VLeuDNAasojnYHdUrC6TBoaiz

Tavonia EvansTavonia Evans10:23 AM
https://colab.research.google.com/drive/1iUf5lWFOdeJ8WOwYS9CJfgT5AoYc0_vx

11:01 AM
Tavonia Evans left 
12:38 PM
Tavonia Evans joined 
Tavonia Evans joined the stage 
Jason Randall SmithJason Randall Smith12:45 PM
https://colab.research.google.com/drive/1OdfJ5aQYmKGSWEzOoXyK7xaONIFFXmiJ

Jason Randall SmithJason Randall Smith1:30 PM
https://colab.research.google.com/drive/1kqkgyq_VLeuDNAasojnYHdUrC6TBoaiz

Jason Randall SmithJason Randall Smith1:49 PM
def entropy(labels): """ Computes entropy of 0-1 vector. """ n_labels = len(labels)

if n_labels <= 1:
    return 0

counts = np.bincount(labels)
probs = counts[np.nonzero(counts)] / n_labels
n_classes = len(probs)

if n_classes <= 1:
    return 0
return - np.sum(probs * np.log(probs)) / np.log(n_classes)
Tavonia EvansTavonia Evans1:50 PM
<code>def entropy(labels): """ Computes entropy of 0-1 vector. """ n_labels = len(labels)</code>

Jason Randall SmithJason Randall Smith1:51 PM
Fastest way to compute entropy in Python

https://stackoverflow.com/questions/15450192/fastest-way-to-compute-entropy-in-python

2:05 PM
Jason Randall Smith left 
2:33 PM
Micah Gerelnyam joined 
Micah Gerelnyam joined the stage 
Micah GerelnyamMicah Gerelnyam2:34 PM
1 sec

Jason Randall Smith gave a thumbs up 
Micah Gerelnyam joined 
Micah Gerelnyam joined the stage 
Micah Gerelnyam left 
Jason Randall SmithJason Randall Smith2:37 PM
I'm here.

Jason Randall Smith joined 
Jason Randall Smith joined the stage 
Jason Randall SmithJason Randall Smith2:38 PM
Write a function to compute entropy given an input of a sequence of probabilities.

https://colab.research.google.com/gist/JSON-Project/42c9cc17cffb6a5c64dbcb9f389f8cf9/json-tavonia-day-29-lectures-1-2-assignment.ipynb

Tavonia EvansTavonia Evans2:47 PM
tennis = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/tennis_decision.csv')

Micah GerelnyamMicah Gerelnyam2:50 PM
import numpy as np

#p = [0.7, 0.6, 0.5]

df = pd.read('file.csv')

empty_list = []

for i in df.column_some: empty_list.append( i * np.log2(i) )

shannon = - sum(empty_list )

Micah Gerelnyam left the stage 
Micah Gerelnyam left 
Jason Randall SmithJason Randall Smith2:58 PM
zinport nunpy as np

3

a

up = [6.7, 0.6, 0.5]

5

Idf = pd.read('ftle.csv')

3

D

3enpty_ltst = []

l

zfor 1. in df.colunn_sone:

3 enpty_1tst.append( t ' np.1092(t) ) 1

Sshannon = - Isun(enpty_‘l.tst )

Tavonia EvansTavonia Evans2:59 PM
https://colab.research.google.com/drive/1kqkgyq_VLeuDNAasojnYHdUrC6TBoaiz